1\. Write a function that converts number representation (bin<->dec<->hex)

In [42]:
def bin_to_int(n) : #assuming x>=0
    if n == 0 : return 0
    l=n.bit_length()
    x=0
    for i in range(l) :
        x += (((n & 2**i) >> i)*2**i)
    return x

print(bin_to_int(0b1100100)) #100

def int_to_bin(n) : #cfr https://www.rapidtables.com/convert/number/decimal-to-binary.html per l'algoritmo
    q=n/2
    #r=bin(n%2) #così r risulta essere di tipo str; faccio a meno di bin(), tanto << ecc. funzionano con i numeri binari
    r=n%2 #in base 10 questo fa o 1 o 0 (n dispari/pari)
    #x=bin(0)
    x=0
    i=0
    while(True) :
        r = r << i
        x+=r
        #print("q =",q)
        #print("r =",r)
        #print("x =",x)
        if q == 0 : break #in alternativa si potrebbe mettere questa condizione dentro il while o qualcosa del genere
        else :
            #r=bin(q%2)
            r=int(q%2)
            q=q//2#ho già valutato il resto, quindi posso ammazzarlo con la divisione intera e/o il casting a int
            i+=1
    return format(x,"b")

print(int_to_bin(100))

#in alternativa posso fare qualcosa di dispendioso con le liste
def int_to_bin2(n) :
    q=n/2
    r=n%2
    x=list()
    while(True) :
        x.append(r)
        #print(x)
        if q == 0 : break
        else :
            r=int(q%2)
            q=q//2
    x.reverse()#ho aggiunto le cifre al contrario
    return "".join([str(i) for i in x])

print(int_to_bin2(100))



100
1100100
1100100


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [45]:
def bin_to_single(x) :
    if x.bit_length() != 32 : #i numeri binari in python sono di tipo int
    #if len(b) != 32 :
        #print("The provided binary number is not a 32 bit word")
        #return None
        b = x + (1 << 31) #se non copio da un'altra parte l'argomento della funzione questo verrà modificato anche fuori dalla funzione...
        s=1 #aggiungo 100... in modo da "costringerlo" ad effettuare uno zero padding; sistemo a mano il segno visto che in questo modo il primo bit viene scambiato
    else :
        b=x
        s=-1
    #n = b << 1
    e = b >> 23 #niente indexing = servono shift e mask
    e = e & 255
    f=0
    for i in range(23) :
        n = b & 2**i
        n = n >> i
        if n != 0 :#potrei mettere una formula generale ma rischio di sommare 0*2^cose. Non so se pesi di più fare if o operazioni con numeri piccoli
            f+=2**(i-23)
    f+=1
    print("s =",s)
    print("e =",e)
    print("1.f =",f)
    return s*f*(2**(e-127))

x=0b11000000101100000000000000000000 #-5.5
x=bin_to_single(x)
print(x)
print("\n")

def binstr_to_single(b) :
    b_str=b[2:]#se gli passi 0b... Se la stringa non contiene 0b questa riga non serve
    #print(b_str)
    if len(b_str) != 32 :
        print("The provided binary number is not a 32 bit word")
        return None
    s=b_str[0]
    if s == "0" : s=1
    else : s=-1
    e=int(b_str[1:9],2)
    m=b_str[9:]
    print("s =",s)
    print("e =",e)
    #print("m =",m)
    f=1+sum([int(m[i])*2**(-(i+1)) for i in range(23)])
    print("1.f =",f)
    n=s*f*2**(e-127)
    #print(n)
    return n

x=binstr_to_single("0b11000000101100000000000000000000")
print(x)


print("\n")
y=0b00000011111000000000000000000000 #1.316554e-36
#y=int(format(y,"#032b"),2)
print(bin_to_single(y))
print("\n")
print(binstr_to_single("0b00000011111000000000000000000000"))

s = -1
e = 129
1.f = 1.375
-5.5


s = -1
e = 129
1.f = 1.375
-5.5


s = 1
e = 7
1.f = 1.75
1.316553672920962e-36


s = 1
e = 7
1.f = 1.75
1.316553672920962e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [2]:
from math import inf

x=1.0
i=0
while x != 0 :
    x/=2
    i+=1
#print(i)
print("underflow: 2^(-%d)"%i)

x=1.0
i=0
while x != inf :
    x*=2
    i+=1
print("overflow: 2^(%d)"%i)

underflow: 2^(-1075)
overflow: 2^(1024)


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [6]:
x=y=1.0
i=0
while x+y != x :
    y/=2
    i+=1
print("Smallest significant increment: 2^(-%d)"%i)


Smallest significant increment: 2^(-53)


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [4]:
from math import sqrt
#according to desmos appoximate solutions are 0 and -1e6

def quad_eq_sol1(a,b,c) :
    x1=(-b+sqrt(b*b-4*a*c))/(2*a)
    x2=(-b-sqrt(b*b-4*a*c))/(2*a)
    return x1,x2

print(quad_eq_sol1(0.001,1000,0.001))

def quad_eq_sol2(a,b,c) :
    x1=(2*c)/(-b-sqrt(b*b-4*a*c))
    x2=(2*c)/(-b+sqrt(b*b-4*a*c))
    return x1,x2
print("\n")
print(quad_eq_sol2(0.001,1000,0.001))

#https://people.csail.mit.edu/bkph/articles/Quadratics.pdf
#vedi anche https://math.stackexchange.com/questions/866331/numerically-stable-algorithm-for-solving-the-quadratic-equation-when-a-is-very
def quad_eq_sol_ok(a,b,c) :
    x1_1,x2_1=quad_eq_sol1(a,b,c)
    x1_2,x2_2=quad_eq_sol2(a,b,c)#dispendioso ma più leggibile di return quad...[0] o quad...[1] 
    if b < 0 : return x2_2,x1_1
    else : return x1_2,x2_1

print("\n")
print(quad_eq_sol_ok(0.001,1000,0.001))


(-9.999894245993346e-07, -999999.999999)


(-1.000000000001e-06, -1000010.5755125057)


(-1.000000000001e-06, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [8]:
def f(x) :
    return x*(x-1)

def Df(x,delta) :
    return (f(x+delta)-f(x))/delta

def der_f(x) :
    return 2*x-1

deltas = [10**(-i) for i in range(15) if i%2 == 0]
print("f(1) (exact result) =",der_f(1))
approx_der=list(map(lambda delta: Df(1,delta), deltas))
print("delta vs approximated f'(1):",list(zip(deltas,approx_der)))

f(1) (exact result) = 1
delta vs approximated f'(1): [(1, 2.0), (0.01, 1.010000000000001), (0.0001, 1.0000999999998899), (1e-06, 1.0000009999177333), (1e-08, 1.0000000039225287), (1e-10, 1.000000082840371), (1e-12, 1.0000889005833413), (1e-14, 0.9992007221626509)]


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [41]:
from math import pi,sqrt
import timeit

def f(x) :
    return sqrt(1-x*x)

def I(N) :
    h=2/N #lo calcolo una tantum
    #return N,sum([h*f(i) for i in range(-1,1+h,h)])
    return N,sum([h*f(x) for x in [i*h for i in range(-int(1/h),int(1/h+1),1)]])

exact_int=pi/2
print("exact result =",exact_int)
N,integral=I(100)
print("approximated result with %d slices ="%N,integral)


for n in range(100,1001,100) :
    N,integral=I(n)
    print("approx. int. with %d slices ="%N,integral)
    print("percentual difference between app./exact result =",100*abs(integral-exact_int)/exact_int)
    print("elapsed time with %d slices (seconds):"%N)
    def time_function() : return I(n) #not very elegant but easy to understand and especially to implement with timeit.timeit(), which requires a statement
    #%timeit time_function()
    #print(timeit.timeit(time_function, number=1))#just 1 execution per n in order to save time at the expense of measurement accuracy
    print(format(timeit.timeit(time_function, number=1), "f")) #this works fine if we want a less detailed but easier to read output, which is probably better considering the low accuracy of our measurements
    print("\n")

#no use in implementing a while(time<1 minute) : increment n, with N=100 we have a result with an error less than 0.004% and an execution time of less than 1/1000th of a second

exact result = 1.5707963267948966
approximated result with 100 slices = 1.5691342555492502
approx. int. with 100 slices = 1.5691342555492502
percentual difference between app./exact result = 0.1058107418062065
elapsed time with 100 slices (seconds):
0.000045


approx. int. with 200 slices = 1.5702085158895223
percentual difference between app./exact result = 0.03742120447745672
elapsed time with 200 slices (seconds):
0.000085


approx. int. with 300 slices = 1.5704763303905234
percentual difference between app./exact result = 0.020371603811050517
elapsed time with 300 slices (seconds):
0.000134


approx. int. with 400 slices = 1.5705884724197634
percentual difference between app./exact result = 0.01323242049828744
elapsed time with 400 slices (seconds):
0.000184


approx. int. with 500 slices = 1.5706475937537707
percentual difference between app./exact result = 0.009468639478506844
elapsed time with 500 slices (seconds):
0.000191


approx. int. with 600 slices = 1.5706831794256648
per